In [1]:
import tensorflow as tf
from model import LabelEncoder, Anchor
from utils import data_preprocess, data_generator
import numpy as np 
import tensorflow_datasets as tfds
import model.Model as Model
import model.Loss as Loss
import os


data_dir = r'F:\dataset\kitti\data_object_image_2\training'


In [2]:
# get data list
img_list = tf.io.gfile.glob(f'{data_dir}\image_2\*.png')
label_list = tf.io.gfile.glob(f'{data_dir}\label_2\*.txt')
img_list.sort()
label_list.sort()

dataset = data_generator.get_dataset(img_list, label_list)

In [3]:
# img, labels, classes = data_preprocess.prepare_data(img_list[0],label_list[0])
# ax = data_preprocess.visualize_detections(img, labels)

In [4]:
model_dir = 'retinanet/'
label_encoder = LabelEncoder.LabelEncoder()

num_classes = 8
batch_size = 2

learning_rate = [2.5e-06, 0.000625, 0.00125, 0.0025, 0.00025, 2.5e-05]
learning_rate_boundaries = [125, 250, 500, 240000, 360000]
learning_rate_fn = tf.optimizers.schedules.PiecewiseConstantDecay(
    boundaries=learning_rate_boundaries, values = learning_rate
)


In [5]:
resnet50_backbone = Model.get_backbone()
loss_fn = Loss.RetinaNetLoss(num_classes)
model = Model.RetinaNet(num_classes, resnet50_backbone)

optimizer = tf.optimizers.SGD(learning_rate=learning_rate_fn, momentum=0.9)
model.compile(loss=loss_fn, optimizer= optimizer)



In [6]:
callbacks_list = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(model_dir, "weights" + "_epoch_{epoch}"),
        monitor="loss",
        save_best_only=False,
        save_weights_only=True,
        verbose=1,
    )
]

In [7]:
dataset = dataset.shuffle(512).prefetch(2).batch(1)
model.fit(
    dataset.take(100),
    epochs=10,
    callbacks = callbacks_list,
)

Epoch 1/10


KeyboardInterrupt: 